# About this notebook

This notebook is for test running the data pipeline and  as a sandbox for testing new functions that we are adding to data pipeline.

In [20]:
# import packages
import numpy as np
import pandas as pd
import argparse
import os

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys

sys.path.append("../../hourly-egrid/")

# import local modules
import src.data_cleaning as data_cleaning
import src.gross_to_net_generation as gross_to_net_generation
import src.load_data as load_data
import src.impute_hourly_profiles as impute_hourly_profiles
import src.eia930 as eia930

# Run the pipeline

In [ ]:
%cd ../src
%run data_pipeline --year 2020

In [21]:
%cd ../src
%run data_pipeline --small SMALL --year 2020

a:\GitHub\hourly-egrid\src
PUDL data already downloaded
egrid2019_data.xlsx already downloaded
egrid2020_data.xlsx already downloaded
BALANCE_2020_Jan_Jun data already downloaded
BALANCE_2020_Jul_Dec data already downloaded
INTERCHANGE_2020_Jan_Jun data already downloaded
INTERCHANGE_2020_Jul_Dec data already downloaded
EBA_elec.csv already downloaded
EBA_raw.csv already downloaded
epa_eia_crosswalk.csv already downloaded
Cleaning EIA-923 data


# Investigate missing data

In [ ]:
year = 2020
path_prefix = ''
eia923_allocated = pd.read_csv(f'../data/outputs/{path_prefix}eia923_allocated_{year}.csv', parse_dates=['report_date'])
plant_frame = pd.read_csv(f"../data/outputs/{path_prefix}plant_static_attributes_{year}.csv")
residual_profiles = pd.read_csv(f"../data/outputs/{path_prefix}residual_profiles_{year}.csv")
eia923_allocated = eia923_allocated.merge(plant_frame, how='left', on='plant_id_eia')

In [ ]:
# 11. Assign hourly profile to monthly data
print("Assigning hourly profile to monthly EIA-923 data")
# create a separate dataframe containing only the generators for which we do not have CEMS data
monthly_eia_data_to_shape = eia923_allocated[
    (eia923_allocated["hourly_data_source"] == "eia")
    & ~(eia923_allocated["fuel_consumed_mmbtu"].isna())
]

In [ ]:
monthly_data_to_shape[monthly_data_to_shape['energy_source_code'] == 'SUN']

In [ ]:
hourly_profiles = impute_hourly_profiles.impute_missing_hourly_profiles(
    monthly_eia_data_to_shape, residual_profiles, year
)

In [ ]:
hourly_profiles['report_date'] = pd.to_datetime(hourly_profiles['report_date'])

In [ ]:
hourly_profiles[(hourly_profiles['fuel_category'] == 'solar') & (hourly_profiles['ba_code'] == 'ISNE')]

In [ ]:
px.line(hourly_profiles[hourly_profiles['fuel_category'] == 'solar'], x='datetime_local', y='profile', color='ba_code')

In [ ]:
hourly_profiles = impute_hourly_profiles.convert_profile_to_percent(hourly_profiles)

In [ ]:
px.line(hourly_profiles[hourly_profiles['fuel_category'] == 'solar'], x='datetime_local', y='profile', color='ba_code')

In [ ]:

shaped_eia_data = impute_hourly_profiles.shape_monthly_eia_data_as_hourly(
    monthly_eia_data_to_shape, hourly_profiles
)

In [ ]:
# specify columns containing monthly data that should be distributed to hourly
columns_to_shape = [
    "net_generation_mwh",
    "fuel_consumed_mmbtu",
    "fuel_consumed_for_electricity_mmbtu",
    "co2_mass_lb",
    "ch4_mass_lb",
    "n2o_mass_lb",
    "nox_mass_lb",
    "so2_mass_lb",
    "co2_mass_lb_for_electricity",
    "ch4_mass_lb_for_electricity",
    "n2o_mass_lb_for_electricity",
    "nox_mass_lb_for_electricity",
    "so2_mass_lb_for_electricity",
    "co2_mass_lb_adjusted",
    "ch4_mass_lb_adjusted",
    "n2o_mass_lb_adjusted",
    "nox_mass_lb_adjusted",
    "so2_mass_lb_adjusted",
]

# group eia data by plant
shaped_monthly_data = (
    monthly_eia_data_to_shape.groupby(
        [
            "plant_id_eia",
            "subplant_id",
            "report_date",
            "plant_primary_fuel",
            "hourly_data_source",
            "fuel_category",
            "fuel_category_eia930",
            "ba_code",
            "ba_code_physical",
            "state",
            "distribution_flag",
        ],dropna=False)
    .sum()
    .reset_index()
)

In [ ]:
monthly_eia_data_to_shape.plant_primary_fuel.unique()

In [ ]:
shaped_monthly_data.plant_primary_fuel.unique()

In [ ]:


# merge the hourly profiles into each plant-month
shaped_monthly_data = shaped_monthly_data.merge(
    hourly_profiles, how="left", on=["report_date", "fuel_category", "ba_code"]
)

# plant-months where there is negative net generation, assign a flat profile
shaped_monthly_data.loc[
    shaped_monthly_data["net_generation_mwh"] < 0, "profile"
] = 1 / (shaped_monthly_data["report_date"].dt.daysinmonth * 24)
shaped_monthly_data.loc[
    shaped_monthly_data["net_generation_mwh"] < 0, "profile_method"
] = "flat_negative_generation"

# shape the data
for column in columns_to_shape:
    shaped_monthly_data[column] = (
        shaped_monthly_data[column] * shaped_monthly_data["profile"]
    )
shaped_monthly_data = shaped_monthly_data.drop(columns=["profile"])

# re order the columns
column_order = [
    "plant_id_eia",
    "subplant_id",
    "datetime_local",
    "datetime_utc",
    "report_date",
    "net_generation_mwh",
    "fuel_consumed_mmbtu",
    "fuel_consumed_for_electricity_mmbtu",
    "co2_mass_lb",
    "ch4_mass_lb",
    "n2o_mass_lb",
    "nox_mass_lb",
    "so2_mass_lb",
    "co2_mass_lb_for_electricity",
    "ch4_mass_lb_for_electricity",
    "n2o_mass_lb_for_electricity",
    "nox_mass_lb_for_electricity",
    "so2_mass_lb_for_electricity",
    "co2_mass_lb_adjusted",
    "ch4_mass_lb_adjusted",
    "n2o_mass_lb_adjusted",
    "nox_mass_lb_adjusted",
    "so2_mass_lb_adjusted",
    "profile_method",
    "hourly_data_source",
]
shaped_monthly_data = shaped_monthly_data[column_order]

In [ ]:
shaped_eia_data[(shaped_eia_data['fuel_category'] == 'solar')]

# Combine all plant data together

In [ ]:
# load data from csv
year = 2020
path_prefix = ''
cems = pd.read_csv(f'../data/outputs/{path_prefix}cems_{year}.csv')
partial_cems = pd.read_csv(f'../data/outputs/{path_prefix}partial_cems_scaled_{year}.csv')
shaped_eia_data = pd.read_csv(f'../data/outputs/{path_prefix}shaped_eia923_data{year}.csv')
plant_frame = pd.read_csv(f"../data/outputs/{path_prefix}plant_static_attributes.csv")


In [ ]:
# check that none of the sources have overlapping subplant-months
columns_to_check_for_duplicates = ['plant_id_eia','subplant_id','report_date']
cems_subplant_months = cems[columns_to_check_for_duplicates].drop_duplicates()
cems_subplant_months['cems'] = 1
partial_cems_subplant_months = partial_cems[columns_to_check_for_duplicates].drop_duplicates()
partial_cems_subplant_months['partial_cems'] = 1
shaped_eia_subplant_months = shaped_eia_data[columns_to_check_for_duplicates].drop_duplicates()
shaped_eia_subplant_months['shaped_eia'] = 1

data_source_overlap = cems_subplant_months.merge(partial_cems_subplant_months, how='outer',on=columns_to_check_for_duplicates).merge(shaped_eia_subplant_months, how='outer',on=columns_to_check_for_duplicates).fillna(0)
data_source_overlap

# check that there is no overlap between shaped eia and cems data
data_source_overlap[(data_source_overlap.shaped_eia == 1) & ((data_source_overlap.cems == 1) | (data_source_overlap.partial_cems == 1))]

# check for overlap between cems and partial cems data
data_source_overlap[(data_source_overlap.cems == 1) & (data_source_overlap.partial_cems == 1)]

In [ ]:
cems = data_cleaning.filter_unique_cems_data(cems, partial_cems)

In [ ]:
combined_plant_data = data_cleaning.combine_subplant_data(cems, partial_cems, shaped_eia_data)
combined_plant_data

In [ ]:
ba_tz = load_data.load_ba_reference()[["ba_code", "timezone_local"]]


In [ ]:
ba_table.columns

In [ ]:
ba_tz = load_data.load_ba_reference()[["ba_code", "timezone_local"]]
generated_emission_rate_columns = [
    "generated_co2_rate_lb_per_mwh_for_electricity",
    "generated_ch4_rate_lb_per_mwh_for_electricity",
    "generated_n2o_rate_lb_per_mwh_for_electricity",
    "generated_nox_rate_lb_per_mwh_for_electricity",
    "generated_so2_rate_lb_per_mwh_for_electricity",
    "generated_co2_rate_lb_per_mwh_adjusted",
    "generated_ch4_rate_lb_per_mwh_adjusted",
    "generated_n2o_rate_lb_per_mwh_adjusted",
    "generated_nox_rate_lb_per_mwh_adjusted",
    "generated_so2_rate_lb_per_mwh_adjusted",
]

for ba in list(ba_fuel_data.ba_code.unique()):

    # filter the data for a single BA
    ba_table = ba_fuel_data[ba_fuel_data["ba_code"] == ba].drop(columns="ba_code")

    # convert the datetime_utc column back to a datetime
    ba_table["datetime_utc"] = pd.to_datetime(ba_table["datetime_utc"], utc=True)

    # calculate a total for the BA
    ba_total = ba_table.groupby(["datetime_utc"]).sum()[data_columns].reset_index()
    ba_total["fuel_category"] = "total"

    # concat the totals to the fuel-specific totals
    ba_table = pd.concat([ba_table, ba_total], axis=0, ignore_index=True)

    # round all values to one decimal place
    ba_table = ba_table.round(2)

    for emission_type in ["_for_electricity", "_adjusted"]:
        for emission in ["co2", "ch4", "n2o", "nox", "so2"]:
            ba_table[f"generated_{emission}_rate_lb_per_mwh{emission_type}"] = (
                (
                    ba_table[f"{emission}_mass_lb{emission_type}"]
                    / ba_table["net_generation_mwh"]
                )
                .fillna(0)
                .replace(np.inf, np.NaN)
                .replace(-np.inf, np.NaN)
            )

    # create a local datetime column
    local_tz = ba_tz.loc[ba_tz["ba_code"] == ba, "timezone_local"].item()
    ba_table["datetime_local"] = ba_table["datetime_utc"].dt.tz_convert(local_tz)

    # re-order columns
    ba_table = ba_table[['fuel_category','datetime_local','datetime_utc'] + data_columns + generated_emission_rate_columns]

    # export to a csv
    ba_table.to_csv(
        f"../data/results/{path_prefix}power_sector_data/{ba}.csv", index=False
    )



In [ ]:
for ba in list(ba_fuel_data.ba_code.unique()):

    # filter the data for a single BA
    ba_table = ba_fuel_data[ba_fuel_data["ba_code"] == ba].drop(columns="ba_code")

    # convert the datetime_utc column back to a datetime
    ba_table["datetime_utc"] = pd.to_datetime(ba_table["datetime_utc"], utc=True)

    # calculate a total for the BA
    ba_total = (
        ba_table.groupby(["datetime_utc"])
        .sum()[data_columns]
        .reset_index()
    )
    ba_total["fuel_category"] = "total"

    # concat the totals to the fuel-specific totals
    ba_table = pd.concat([ba_table, ba_total], axis=0, ignore_index=True)

    # round all values to one decimal place
    ba_table = ba_table.round(1)

    for emission_type in ['_for_electricity','_adjusted']:
        for emission in ['co2','ch4','n2o','nox','so2']:
            ba_table[f"generated_{emission}_rate_lb_per_mwh{emission_type}"] = (
                    (ba_table[f"{emission}_mass_lb{emission_type}"] / ba_table["net_generation_mwh"])
                    .fillna(0)
                    .replace(np.inf, np.NaN).replace(-np.inf, np.NaN)
                )

    # export to a csv
    ba_table.to_csv(f"../data/results/{path_prefix}power_sector_data/{ba}.csv")

# Test new functions

In [ ]:
year = 2020
path_prefix = ''
cems = pd.read_csv(f'../data/outputs/{path_prefix}cems_{year}.csv', parse_dates=['operating_datetime_utc','report_date'])
eia923_allocated = pd.read_csv(f'../data/outputs/{path_prefix}eia923_allocated_{year}.csv', parse_dates=['report_date'])

In [ ]:
cems.energy_source_code.unique()

In [ ]:
cems[cems['energy_source_code'] == 'MSW']

In [ ]:

px.line(cems[cems['energy_source_code'] == 'BLQ'], x='operating_datetime_utc', y='fuel_consumed_mmbtu', color='cems_id')

In [ ]:
plant_frame = pd.read_csv(f"../data/outputs/{path_prefix}plant_static_attributes.csv")
eia923_allocated = eia923_allocated.merge(plant_frame, how='left', on='plant_id_eia')

In [ ]:
# 11. Assign hourly profile to monthly data
print('Assigning hourly profile to monthly EIA-923 data')
# create a separate dataframe containing only the generators for which we do not have CEMS data
monthly_eia_data_to_distribute = eia923_allocated[
    (eia923_allocated["hourly_data_source"] == "eia")
    & ~(eia923_allocated["fuel_consumed_mmbtu"].isna())
]
# load profile data and format for use in the pipeline
# TODO: once this is in the pipeline (step 10), may not need to read file
hourly_profiles = pd.read_csv(
    "../data/outputs/residual_profiles.csv", parse_dates=["report_date"]
)


In [ ]:
available_profiles = hourly_profiles[['ba_code','fuel_category']].drop_duplicates()
ba_fuel_to_distribute = monthly_eia_data_to_distribute[['ba_code','fuel_category']].drop_duplicates().dropna()
missing_profiles = ba_fuel_to_distribute.merge(available_profiles, how='outer', on=['ba_code','fuel_category'], indicator='source')
missing_profiles = missing_profiles[missing_profiles.source == 'left_only']
missing_profiles.sort_values(by=['fuel_category','ba_code'])

In [ ]:
hourly_profiles = hourly_profiles.load_hourly_profiles(monthly_eia_data_to_distribute, year)

In [ ]:
print(hourly_profiles[['ba_code','fuel_category','profile_method']].drop_duplicates().pivot_table(index='fuel_category',columns='profile_method', aggfunc='count').fillna(0).astype(int))

# investigate profile shapes

In [ ]:
import plotly.express as px
import src.eia930 as eia930

In [ ]:
hydro_demand = load_data.load_raw_eia930_data(year, 'BALANCE')
hydro_demand = hydro_demand[["Balancing Authority","datetime_utc","Demand (MW)","Net Generation (MW)","Net Generation (MW) from Hydropower and Pumped Storage",]]

In [ ]:
bas_with_no_hydro = hydro_demand.groupby("Balancing Authority").sum().reset_index()
bas_with_no_hydro = list(bas_with_no_hydro.loc[(bas_with_no_hydro["Net Generation (MW) from Hydropower and Pumped Storage"] == 0),"Balancing Authority"])

In [ ]:
hydro_demand = hydro_demand[~hydro_demand['Balancing Authority'].isin(bas_with_no_hydro)]

In [ ]:
px.line(hydro_demand, x='datetime_utc', y='Net Generation (MW) from Hydropower and Pumped Storage', color='Balancing Authority')

In [ ]:
hydro_corr = hydro_demand.groupby("Balancing Authority")[["Demand (MW)","Net Generation (MW) from Hydropower and Pumped Storage"]].corr().reset_index()
hydro_corr = hydro_corr[hydro_corr['level_1'] == 'Demand (MW)'].drop(columns=["Demand (MW)","level_1"])
hydro_corr

In [ ]:
cleaned_930 = eia930.load_chalendar_for_pipeline(
    "../data/outputs/EBA_adjusted_elec.csv", year=year
)

In [ ]:
cleaned_930.fuel_category.unique()

In [ ]:
fuel = 'other'

data_to_plot = cleaned_930[cleaned_930['fuel_category'] == fuel]

px.line(data_to_plot, x='datetime_local', y='net_generation_mwh_930', color='ba_code')

In [ ]:
data_to_plot

In [ ]:
data_to_plot.pivot(index='datetime_local', columns='ba_code', values='net_generation_mwh_930').corr()